In [1]:
import numpy as np
import theano.tensor as T
import theano
from theano import function
from theano import pp

In [9]:
a = T.dscalar('x')
b = T.dscalar('y')
z = a + b
f = function([a, b], z)

In [14]:
x = T.dvector('x')
y = x ** 2
J, updates = theano.scan(lambda i, y, x : T.grad(y[i], x), sequences=T.arange(y.shape[0]), non_sequences=[y, x])

In [20]:
pp(y)

'(x ** TensorConstant{2})'

In [14]:
k = T.iscalar("k")
A = T.vector("A")

# Symbolic description of the result
result, updates = theano.scan(fn=lambda prior_result, A: prior_result * A,
                              outputs_info=T.ones_like(A),
                              non_sequences=A,
                              n_steps=k)

# We only care about A**k, but scan has provided us with A**1 through A**k.
# Discard the values that we don't care about. Scan is smart enough to
# notice this and not waste memory saving them.
final_result = result[-1]

# compiled function that returns A**k
power = theano.function(inputs=[A,k], outputs=final_result, updates=updates)

print(power(range(10),2))
print(power(range(10),4))

[ 0.  1.  4.  9. 16. 25. 36. 49. 64. 81.]
[0.000e+00 1.000e+00 1.600e+01 8.100e+01 2.560e+02 6.250e+02 1.296e+03
 2.401e+03 4.096e+03 6.561e+03]


In [ ]:
import numpy as np

<strong> Hessian test
        </strong>
 \begin{equation}
 y = x_1^3 \cdot x_2^4
  \end{equation}
  
  Hessian: 
   \begin{equation}
   \begin{bmatrix}
    \frac{\partial^2 y}{\partial x_1^2} & \frac{\partial^2 y}{\partial x_1 \partial x_2} \\
    \frac{\partial^2 y}{\partial x_2 \partial x_1} & \frac{\partial^2 y}{\partial x_2^2} 
  \end{bmatrix}
  \end{equation}
  

In [30]:
x = T.dvector('x')
y = x[0]** 3 * x[1] **4
cost = y.sum()
hy = T.hessian(cost, x)
f = theano.function([x] , hy,updates=updates)
f([2,1])

array([[12., 48.],
       [48., 96.]])